<a href="https://colab.research.google.com/github/neha-duggirala/EY-Recommendation-Engine/blob/master/EY_Badge_Recommendation_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/My Drive/datasets/EY Hackathon/GDS_EY_Badges.xlsx'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pandas_profiling
'Done Importing'

'Done Importing'

In [3]:
originalDf = pd.read_excel(path)
data = pd.read_excel(path)
data.drop(['Requestor GPN','GUI','LastName','FirstName','EmailAddress','Country','Badge Status'],axis=1,inplace=True)
data['Time Stamp'] = data['Badge earned'] - data['Initiate a badge date']
data['Time Stamp'].fillna(pd.Timedelta(seconds=0),inplace=True)
data.drop(['Badge earned','Initiate a badge date'],axis=1,inplace=True)
data.head()

,EmployeeStatusDesc,Rank Name,Badge Type,Domain,Sub Domain,Badge Classification,GDS SL,Time Stamp
0,Terminated,Rank 9,Bronze,Analytics,Data science,TECHNOLOGY,SL 26,666 days
1,Terminated,Rank 9,Bronze,Emerging Technology,Robotic process automation,TECHNOLOGY,SL 26,4 days
2,Active,Rank 9,Bronze,Emerging Technology,Blockchain,TECHNOLOGY,SL 20,37 days
3,Active,Rank 1,Bronze,Analytics,Data visualization,TECHNOLOGY,SL 8,2 days
4,Active,Rank 9,Bronze,Analytics,Data visualization,TECHNOLOGY,SL 25,10 days


In [4]:
def buildMatrix(row_feature,col_feature):
  rowCol = []
  for rowEach in data[row_feature].unique():
    row =[]
    rowEachDF=data[data[row_feature]==rowEach]
    for colEach in data[col_feature].unique():
      row.append(rowEachDF[rowEachDF[col_feature].str.contains(colEach)].shape[0])
    rowCol.append(row)
  return rowCol

In [5]:
def dataPreparation(data):
  badgeClassification={}
  for domain in data['Domain'][data['Badge Classification'].str.contains('LEADER')].unique():
    badgeClassification[domain]='LEADERSHIP'
  for domain in data['Domain'][data['Badge Classification'].str.contains('BUSINESS')].unique():
    badgeClassification[domain]='BUSINESS'
  for domain in data['Domain'][data['Badge Classification'].str.contains('TECH')].unique():
    badgeClassification[domain]='TECHNOLOGY'

  levelDomainMatrix = buildMatrix('GDS SL','Domain')
  subDomainMatrix = buildMatrix('Domain','Sub Domain')
  domainSubDomainDict = {}
  for domain in data.Domain.unique():
    df = data[data['Domain'].str.contains(domain)]
    domainSubDomainDict[domain]=df['Sub Domain'].unique().tolist()
  return badgeClassification,domainSubDomainDict,levelDomainMatrix,subDomainMatrix


In [6]:
#only works for one record
def dropUnnecessaryFeatures(df,axis=0):
  df.drop(['Requestor GPN','GUI','LastName','FirstName','EmailAddress','Country','Badge Status','EmployeeStatusDesc','Badge earned','Initiate a badge date'],axis,inplace=True)


In [7]:
def index(feature,item):
  idx = np.where(data[feature].unique()==item)
  return((idx[0][0]))

In [8]:
# Function returns indices of N largest elements 
def NmaxelementsIdx(list, N): 
    final_list = [] 
    list1 = list[:]
    for i in range(0, N):  
        max1 = 0
          
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j];
                idx1 = list.index(max1)
                  
        list1.remove(max1); 
        final_list.append(idx1)
    return(final_list)

In [47]:
def recommendationEngine(data,currentBadge):
  badgeClassification,domainSubDomainDict,levelDomainMatrix,subDomainMatrix = dataPreparation(data)
  print("******Before truncating the unnecessary features*********** \n",currentBadge)
  dropUnnecessaryFeatures(currentBadge)
  print("*************After removing the unnecessary features******** \n",currentBadge)
  bestOf=2
  idx = index('GDS SL',currentBadge['GDS SL'])
  levelDomainList = levelDomainMatrix[idx]
  domainIdx = NmaxelementsIdx(levelDomainList,bestOf)
  print("***********Domain Recommendations*****")
  print(domainIdx)
  domainRecommendation = [data['Domain'].unique()[idx] for idx in domainIdx]
  print(domainRecommendation)
  print("***********Sub Domain Recommendations********")
  subDomainRecommendation=[]
  subDomainIdx=[]
  for domain in domainIdx:
    subDomainList = subDomainMatrix[domain]
    Idx = NmaxelementsIdx(subDomainList,bestOf)
    subDomainRecommendation.append([data['Sub Domain'].unique()[idx] for idx in Idx])
    subDomainIdx.append(Idx)
  print(subDomainIdx)
  print(subDomainRecommendation)
  print("********Badge Recommendations**********")
  currentBadgeIdx = currentBadgeIdx2 = index('Badge Type',currentBadge['Badge Type'])
  if currentBadgeIdx>4: 
    currentBadgeIdx2-=1 
  else: currentBadgeIdx2+=1
  badgeRecommendationIdx = [currentBadgeIdx,currentBadgeIdx2]
  badgeRecommendation = [data['Badge Type'].unique()[idx] for idx in badgeRecommendationIdx]
  print(badgeRecommendation)
  recommendations = []
  for idx in range(len(domainRecommendation)):
      for eachsubDomain in subDomainRecommendation[idx]:
          if eachsubDomain not in domainSubDomainDict[domainRecommendation[idx]]:
            randomDomain = random.randint(0,len(domainSubDomainDict[domainRecommendation[idx]])-1)
            eachsubDomain = domainSubDomainDict[domainRecommendation[idx]][randomDomain]
          obj = {'Badge Type':"",
                'Domain':domainRecommendation[idx],
                'Sub Domain': eachsubDomain,
                'Badge Classification':badgeClassification[domainRecommendation[idx]],
                }

          recommendations.append(obj)
  sol=[]
  for badge in badgeRecommendation:
    for idx in range(len(recommendations)):
      recommendations[idx]['Badge Type'] = badge
      print(recommendations[idx])
      sol.append(recommendations[idx])


In [48]:
import random
rand = random.randint(0, originalDf.shape[0])
currentBadge = originalDf.iloc[rand]
# random.shuffle(recommendationEngine(data,currentBadge))
recommendationEngine(data,currentBadge)

******Before truncating the unnecessary features*********** 
 Requestor GPN                      GPN 14067
GUI                                      123
LastName                             XY14067
FirstName                                ABC
EmailAddress              abc.xyz@gds.ey.com
EmployeeStatusDesc                    Active
Country                            Country 3
Rank Name                             Rank 9
Badge Type                            Bronze
Badge Status                        Approved
Domain                                Sector
Sub Domain                         Insurance
Badge Classification                BUSINESS
Badge earned             2020-05-02 00:00:00
GDS SL                                 SL 20
Initiate a badge date    2020-04-15 00:00:00
Name: 14066, dtype: object
*************After removing the unnecessary features******** 
 Rank Name                  Rank 9
Badge Type                 Bronze
Domain                     Sector
Sub Domain              In

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3939: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(obj)


In [28]:
domainRecommendation

NameError: ignored